In [1]:
import sys
sys.path.append('../')

import photonqat as pq

In [2]:
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib.pyplot as plt

ガウス操作に光子数測定を組み合わせることで古典計算機で効率的に計算できない操作を行うことが可能  
しかし現実の計測器に光子1個分の分解能を要求するのは現状かなり厳しい  
そこで、光子数を知ることはできないが光子がきたことを検出できる計測器を用いた計算も研究されている

10.1103/PhysRevA.72.033822  
"Conditional generation of arbitrary single-mode quantum states of light by repeated photon subtractions"

測定演算子  
$\Pi_0 = |0\rangle \langle 0|$  
$\Pi_1 = I - |0\rangle \langle 0|$  

$W_{AB}(r_{AB}) = \frac{\sqrt{\rm{det}\Gamma_{AB}}}{\pi^{2}}\exp\{ {-(r_{AB} - d_{AB})^T \Gamma_{AB}(r_{AB} - d_{AB})}\}$

$d_{AB} = (d_A, d_B)^T = S(\mu_A, \mu_B)^T$

$\Gamma_{AB} = \gamma_{AB}^{-1}$

$\gamma_{AB} = S(\gamma_A \oplus I_B)S^T + G$

$S = S_\eta S_{BS} = (I_A \oplus \sqrt{\eta}I_B) S_{BS}$

$G = 0_A \oplus (1-\eta)I_B$

In [57]:
N = 2
G = pq.Gaussian_trans(N)
G.Displace(0, 1+0.5j) # Displacement gate, x to x+2
G.BS(0, 1, np.pi/4) # 50:50 beam splitter
#G.plotGaussianWigner(0) # plot
print(G.V)

eta = 0.5
S_eta = np.eye(2 * N)
g = np.zeros([2 * N, 2 * N])
idx_B = 1
idx_B = 2 * idx_B
S_eta[idx_B, idx_B] = np.sqrt(eta)
g[idx_B, idx_B] = (1 - eta)
S_eta[idx_B+1, idx_B+1] = np.sqrt(eta)
g[idx_B+1, idx_B+1] = (1 - eta)
gamma_AB = np.dot(np.dot(S_eta, G.V), S_eta.T)  + g
dA = G.mean(0)
dB = G.mean(1)

[[1.00000000e+00 0.00000000e+00 1.01465364e-17 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 1.01465364e-17]
 [1.01465364e-17 0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.01465364e-17 0.00000000e+00 1.00000000e+00]]


$W_{AB}$に対応する密度演算子を$\rho_{AB}$とする。  
inefficient detectorがphotonを検出した時のモードAの状態は次式。  
$\rho_{A,out} = \mathrm{Tr_B}[\rho_{AB}(I_A \oplus \Pi_{1,B})]$

POVM演算子はWinger関数でも書ける。  
$W_{\Pi_1}(r) = \frac{1}{2\pi} - \frac{1}{\pi}e^{-x^2 -p^2}$

$\rho_{A,out} $を再びWigner関数に書き直すと以下のようにGaussian Wignerの和となる。  
Gaussian Wignerの和であり、解析的に式がわかっている以上計算は難しくない  
それでもGauss操作だけでは実現できない状態を作れている

$W(r)P = C_1 W_G(r;\Gamma_1, d_1) + C_2 W_G(r;\Gamma_2, d_2)$

$P$; 上のようなモードAが生成される確率

$\Gamma_{AB} = \begin{pmatrix}
\Upsilon_A & \sigma \\
\sigma^T & \Upsilon_B \\
\end{pmatrix}
$

$\Gamma_1 = \Upsilon_A - \sigma\Upsilon_B^{-1}\sigma^T$  
$d_1 = d_A + z_2$  
$C_1 = 1$

$\Gamma_2 = \Upsilon_A - \sigma\tilde{\Upsilon}_B^{-1}\sigma^T$  
$d_2 = d_A + \Gamma_2^{-1}\sigma\tilde{\Upsilon}_B^{-1} d_B  + z_2$  
$C_2 = -2\sqrt{\frac{\mathrm{det}(\Gamma_{AB}) }{\mathrm{det}(\Gamma_{2}) \mathrm{det}(\tilde{\Upsilon}_B)} } \exp[-d_B^T M d_B]$

$\tilde{\Upsilon}_B = \Upsilon_B + I$  
$M = \Upsilon_B \tilde{\Upsilon}_B^{-1} - \tilde{\Upsilon}_B^{-1} \sigma^T \Gamma_2^{-1}\sigma \tilde{\Upsilon}_B^{-1}$

$S_{s} = \begin{pmatrix}
e^{s_{out}} & 0 \\
0 & e^{-s_{out}} \\
\end{pmatrix}
$

anti squeezeする

$W_{out}(r)P = C_1 W_G(r;\Gamma_1^{'}, d_1^{'}) + C_2 W_G(r;\Gamma_2^{'}, d_2^{'})$  
$\Gamma_{1, 2}^{'} = S_s^{-1}\Gamma_{1,2}S_s^{-1}$  
$d_{1, 2}^{'} = S_s d_{1,2}$

$P = C_1 + C_2$

In [63]:
alpha2 = 1 + 1j
z2 = np.sqrt(2) * np.array([np.real(alpha2), np.imag(alpha2)])

Gamma_AB = np.linalg.inv(gamma_AB)
idx_A = 0
Ups_A = Gamma_AB[idx_A:idx_A+2, idx_A:idx_A+2]
Ups_B = Gamma_AB[idx_B:idx_B+2, idx_B:idx_B+2]
sigma = Gamma_AB[idx_A:idx_A+2, idx_B:idx_B+2]

Ups_B_inv = np.linalg.inv(Ups_B)
Ups_Btilde = Ups_B + np.eye(2)
Ups_Btilde_inv = np.linalg.inv(Ups_Btilde)
sigma_T = sigma.T

Gamma_1 = Ups_A - np.dot(np.dot(sigma, Ups_B_inv), sigma_T)
d1 = dA + z2
C1 = 1

Gamma_2 = Ups_A - np.dot(np.dot(sigma, Ups_Btilde_inv), sigma_T)
Gamma_2_inv = np.linalg.inv(Gamma_2)
d2 = dA + np.dot(np.dot(np.dot(Gamma_2_inv, sigma), Ups_Btilde_inv), dB) + z2
det_GammaAB = np.linalg.det(Gamma_AB)
det_Gamma_2 = np.linalg.det(Gamma_2)
det_Ups_Btilde = np.linalg.det(Ups_Btilde)
CC = np.sqrt(det_GammaAB / det_Gamma_2 / det_Ups_Btilde)
M = np.dot(Ups_B, Ups_Btilde_inv) - np.linalg.multi_dot([Ups_Btilde_inv, sigma_T, Gamma_2_inv, sigma, Ups_Btilde_inv])
C2 = -2 * CC * np.exp(np.dot(np.dot(-dB.T, M), dB))

print(sigma)

[[-7.17468466e-18  0.00000000e+00]
 [ 0.00000000e+00 -7.17468466e-18]]


あまり意味がない気がしたので中断